<a href="https://colab.research.google.com/github/ShravanyaShetty01/Analysis-sentiment-in-movie-reviews/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#3
# Load the dataset into pandas
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Data Preprocessing
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Split the data into input and target
X_train = train_data['message']
y_train = train_data['label']
X_test = test_data['message']
y_test = test_data['label']

# Tokenizing and padding the messages for neural network processing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 150
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))


In [ ]:
#4
# Function to predict messages based on the model
def predict_message(pred_text):
    # Convert the message into a sequence of integers
    pred_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_padded = pad_sequences(pred_sequence, maxlen=max_length, padding='post')

    # Make the prediction
    pred_prob = model.predict(pred_padded)

    # If probability is greater than 0.5, it's 'spam', else it's 'ham'
    label = 'spam' if pred_prob >= 0.5 else 'ham'

    # Return the prediction probability and label
    return [float(pred_prob), label]


In [ ]:
# # function to predict messages based on model
# # (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
# def predict_message(pred_text):



#   return (prediction)

# pred_text = "how are you doing today?"

# prediction = predict_message(pred_text)
# print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
